In [365]:
import gpytorch
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
import torch
from tqdm import tqdm
import numpy as np 
from torch.utils.data import TensorDataset, DataLoader

# This will control precision of Symeig / Cholenski Lin Alg Operations
gpytorch.settings.linalg_dtypes(default=torch.float16)
#TODO: How to ensure that all operations, data, etc. follow this precision constraint?

# Sample code for Batch Learning
def get_DataLoaders(train_x, train_y, test_x, test_y):
    train_dataset = TensorDataset(train_x, train_y)
    train_loader = DataLoader(train_dataset, batch_size=500, shuffle=True)

    test_dataset = TensorDataset(test_x, test_y)
    test_loader = DataLoader(test_dataset, batch_size=500, shuffle=False)
    return train_loader, test_loader

def train_and_test_approximate_gp(model_cls, train_x, train_y, test_y, train_loader, test_loader, num_epochs):
    inducing_points = torch.randn(128, train_x.size(-1), dtype=train_x.dtype, device=train_x.device)
    model = model_cls(inducing_points)
    
    # Likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    # Objective -> Variational Inference Uses ELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.numel())
    optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()

    # Training
    model.train()
    likelihood.train()
    
    # TQDM is just a progress bar for training
    epochs_iter = tqdm.notebook.tqdm(range(num_epochs), desc=f"Training {model_cls.__name__}")
    
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            epochs_iter.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()

    # Testing
    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    error = torch.mean(torch.abs(means - test_y.cpu()))
    print(f"Test {model_cls.__name__} MAE: {error.item()}")

In [366]:
# Extends ApproximateGP
class StandardApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        # Needs a Variational Distribution + Variation Strategy
        #TODO: Which ones do we need to implmenet?
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(-2))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy) # Pass strategy upwards
        
        # Then Kernel and Mean as Normal
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [367]:
import math
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [368]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = InducingPointKernel(gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()), inducing_points=torch.empty(1), likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model

In [369]:
def greedy_select_points(model, inducing_point_candidates, train_x, train_y, mll):
    """
    Inplace Addition of Inducing Point
    
    Output: inducing_point_candidates - Tensor | Returns remaining candidate set of inducing points
    """
    # TODO: Fix Potential dimension errors
    # look at .resize(), torch.cat, len(train)
    # TODO: Pop train_x_copy index
    # Get current MLL

    random_indices = np.random.permutation(len(inducing_point_candidates))
    inducing_points = model.covar_module.inducing_points
    
    # Get MLL from current inducing points
    with torch.no_grad():
            output = model(train_x)
            current_model_mll = mll(output, train_y)

    # While we haven't found a point
    for index in random_indices:
        rnd = inducing_point_candidates[index].resize(1) # TODO: Make this better
        
        # Grab a point at random, calculate its likelihood
        temp = torch.cat((inducing_points, rnd),dim=0)
        
        # Update the inducing point kernel
        model.covar_module.inducing_points = torch.nn.Parameter(temp,requires_grad=False)

        # Get MLL for model with candidate inducing point
        with torch.no_grad(): 
            rnd_point_mll = mll(model(train_x), train_y)

        # If we've increased our likelihood, we've found our point
        if rnd_point_mll > current_model_mll:
            # Catch edge case where we grab the last index
            if index == len(inducing_point_candidates) +1:
                return inducing_point_candidates[0:index]
            else:
                return torch.cat((train_x_copy[0:index], train_x_copy[index+1:]),dim=0) 

        
    # If we couldn't increase our likelihood, get rid of the last appended inducing point
    model.covar_module.inducing_points = torch.nn.Parameter(temp[:-1],requires_grad=False)
    return None

In [370]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood,model)
model.train()
likelihood.train()

train_x = train_x.type(torch.float16)
inducing_points = train_x
model.covar_module.inducing_points = torch.nn.Parameter(inducing_points,requires_grad=False)
train_x_copy = train_x.detach().clone()
train_x_copy = greedy_select_points(model,train_x_copy, train_x, train_y, mll)

RuntimeError: You must train on the training inputs!

In [383]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood,model)
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
max_inducing_points = 1000
inducing_point_candidates = train_x.detach().clone()
training_iter = 1000

for i in range(max_inducing_points):
    # If haven't gotten any inducing points, grab a random one
    if len(inducing_point_candidates) == len(train_x):
        random_index = np.random.randint(0, len(train_x))
        first_inducing_point = train_x[random_index].detach().clone().resize(1) # Get
        model.covar_module.inducing_points = torch.nn.Parameter(first_inducing_point, requires_grad=False) # Set
        # Remove Selected Point from candidate set
        inducing_point_candidates = torch.cat((inducing_point_candidates[:random_index], inducing_point_candidates[random_index + 1 :]),dim=0)
        
    elif len(model.covar_module.inducing_points) >= max_inducing_points:
        print(f"Reached limit of inducing points: we have {len(model.covar_module.inducing_points)} points with a maximum of {max_inducing_points}")
        break    
    else:
        inducing_point_candidates = greedy_select_points(model,inducing_point_candidates,train_x,train_y,mll)
        if inducing_point_candidates is not None:
            continue
        else:
            # We've failed to find a point that increases our Likelihood
            break
        
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    mll.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()

    optimizer.step()

In [384]:
model.covar_module.inducing_points

Parameter containing:
tensor([0.0707, 0.7979, 0.0808, 0.5151, 0.7881, 0.1313, 0.3838],
       dtype=torch.float16)

In [385]:
loss

tensor(1.3313, grad_fn=<NegBackward0>)